<a href="https://colab.research.google.com/github/Melikakmm/CLPBgroup17/blob/main/Exercise_4/ex3q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tsfresh

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14
from tsfresh import extract_features

from xgboost import XGBClassifier, plot_tree

%run /content/plot.py

In [2]:
#creating deepcopy of model instances
from copy import deepcopy

#array manipulations
import numpy as np

#dataframe and its manipulations
import pandas as pd

#plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#selected plotting functions
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

#classes for grid search and cross-validation, function for splitting data and evaluating models
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split

from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve

#XGBoost library
import xgboost as xgb

#Python standard libraries
import time
import warnings

#setting default resolution of figures
mpl.rcParams['figure.dpi'] = 200

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import label_binarize

#from sklearn.preprocessing import OneHotEncode




In [3]:
#read data

str0 = 'ts_L60_Z12_A500_DX50_bias5_N10000.dat'
fnamex='DATA/x_'+str0
fnamey='DATA/y_'+str0

x = np.loadtxt(fnamex, delimiter=" ",dtype=float)
N,L = len(x), len(x[0])

#Show_data(x,L,"original data")

# note: here it does not need to be converted to the 3-bit version, a label remains y[i]=0,1,2
y = np.loadtxt(fnamey, dtype=int)
n_class = 3    #  = len(np.unique(y))
labels = [0, 1, 2]
y = label_binarize(y, classes=labels)
print(x)
print(y)
print('data: ',N)

[[ 0.000000e+00 -5.000000e+00  2.300000e+01 ...  1.630000e+02
   1.690000e+02  2.110000e+02]
 [ 1.830000e+02  2.310000e+02  2.350000e+02 ...  6.230000e+02
   6.190000e+02  6.570000e+02]
 [ 5.800000e+02  6.230000e+02  6.780000e+02 ...  8.040000e+02
   7.890000e+02  7.320000e+02]
 ...
 [ 2.977052e+06  2.977068e+06  2.976997e+06 ...  2.977152e+06
   2.977001e+06  2.976968e+06]
 [ 2.977029e+06  2.976953e+06  2.977015e+06 ...  2.977032e+06
   2.976993e+06  2.976984e+06]
 [ 2.976953e+06  2.976987e+06  2.976895e+06 ...  2.977286e+06
   2.977386e+06  2.977349e+06]]
[[1 0 0]
 [0 1 0]
 [0 0 1]
 ...
 [0 1 0]
 [0 0 1]
 [1 0 0]]
data:  10000


In [4]:
#load data into dataframe
df_x = pd.read_csv('DATA/x_ts_L60_Z12_A500_DX50_bias5_N10000.dat',
                    header=None, sep= " ", engine='python')
df_y = pd.read_csv('DATA/y_ts_L60_Z12_A500_DX50_bias5_N10000.dat',
                    header=None, sep='\s\s+', engine='python')
#dropping id columns
#df_x.drop(df_x.columns[-1], axis=1, inplace=True)
#df_y.drop(df_y.columns[-1], axis=1, inplace=True)
#checking x and data types
#df_x.info()
#df_y.info()
#df_x
#df_y
#merge in a single dataframe
frames = [df_y, df_x]
df = pd.concat(frames,axis=1)

df

,0,0,1,2,3,4,5,6,7,8,...,50,51,52,53,54,55,56,57,58,59
0,0,0,-5,23,3,-19,84,158,167,186,...,120,128,145,149,204,275,235,163,169,211
1,1,183,231,235,242,280,327,285,288,307,...,684,663,690,741,668,622,607,623,619,657
2,2,580,623,678,663,597,579,558,521,610,...,785,799,883,811,830,807,807,804,789,732
3,0,727,663,539,596,654,677,685,730,752,...,462,462,416,397,469,534,564,542,546,599
4,1,658,708,689,699,684,712,595,699,684,...,1286,1285,1334,1321,1327,1349,1401,1286,1290,1291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2,2976175,2976207,2976171,2976224,2976316,2976312,2976365,2976388,2976406,...,2976245,2976222,2976334,2976345,2976370,2976395,2976404,2976323,2976395,2976400
9996,0,2976400,2976411,2976363,2976369,2976424,2976459,2976422,2976360,2976343,...,2976886,2976789,2976873,2976904,2976962,2976926,2976968,2977105,2977108,2977031
9997,1,2977052,2977068,2976997,2976904,2976872,2976835,2976909,2976986,2976943,...,2977396,2977435,2977517,2977481,2977536,2977479,2977276,2977152,2977001,2976968
9998,2,2977029,2976953,2977015,2977005,2977081,2977060,2977051,2977002,2977018,...,2976887,2976866,2976866,2976920,2976930,2976970,2976928,2977032,2976993,2976984


In [5]:
#replace non interpretable values with zeroes
df.loc[df[lambda x: x == '?'].any(axis=1), df[lambda x: x == '?'].any(axis=0)]
df.replace(to_replace='?', value=0, inplace=True)



Check the autocorrelation of a real or complex random process is the Pearson correlation between values of the process at different times.

In [6]:
#checking autocorrelations, whether data are time series
f, axes = plt.subplots(29,2,figsize=(18,180))
for i,ax in enumerate(axes):
    plot_acf(df.iloc[:,i], zero=False, ax=ax[0])
    plot_pacf(df.iloc[:,i], zero=False, ax=ax[1])
plt.show()

/usr/local/lib/python3.7/dist-packages/statsmodels/graphics/tsaplots.py:353: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/graphics/tsaplots.py:353: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/graphics/tsaplots.py:353: FutureWarning: The default method 'yw' can produce PACF values outside of the [-1,1] interval. After 0.13, the default will change tounadjusted Yule-Walker ('ywm'). You can use this method now by setting method='ywm'.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/graphics/tsap

In [7]:
#split train and test
#df_train, df_test = train_test_split(df_x, test_size=0.2)

#train test split with randomization performed (although randomization is not necessary)
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,1:61], df.iloc[:,0], test_size=0.20, random_state=122)

print(y_train, y_test)
print(x_train, x_test)

#df_trainx = pd.DataFrame(x_train)
#df_trainy = pd.DataFrame(y_train)
#df_testx = pd.DataFrame(x_test)
#df_testy = pd.DataFrame(y_test)

#df_trainx
#df_trainy
#df_testx
#df_testy

#frames = [df_trainy, df_trainx]
#df_train = pd.concat(frames,axis=1)
#frames1 = [df_testy, df_testx]
#df_test = pd.concat(frames1,axis=1)

#df_test
#df_train


3367    1
1737    0
5246    2
1407    0
5599    1
       ..
694     1
1342    1
6974    2
2442    0
7663    1
Name: 0, Length: 8000, dtype: int64 2596    1
3408    0
6464    2
4649    2
5869    1
       ..
5898    0
2278    1
9857    2
9501    0
5088    0
Name: 0, Length: 2000, dtype: int64
           0        1        2        3        4        5        6        7   \
3367   973609   973694   973688   973740   973771   973771   973719   973755   
1737   495000   494988   494998   494980   495028   494989   494952   495037   
5246  1547556  1547483  1547523  1547532  1547552  1547485  1547545  1547595   
1407   388903   388885   388864   388863   388868   388796   388753   388688   
5599  1652617  1652596  1652571  1652624  1652608  1652601  1652610  1652551   
...       ...      ...      ...      ...      ...      ...      ...      ...   
694    183654   183563   183643   183659   183663   183668   183715   183768   
1342   371807   371824   371877   371833   371835   371893   371883 

Fit a benchmark model before execute the random gridsearch.
Calling fit(), default parameters are obtained and stored for later use and since gidsearchcv take inputs in lists also single parameter values have to be wrapped, calling fit() on gscv assures that, after the cross validation, results are extracted and scores computed and stored in dicts.

**Parameters for tree base learners**

**F1 Score**: is the weighted average of Precision and Recall, this score takes both false positives and false negatives into account.
**Accuracy** works best if false positives and false negatives have similar cost. If the cost of false positives and false negatives are very different, it’s better to look at both Precision and Recall.

**ROC AUC**: a **ROC** curve (receiver operating characteristic curve) is a graph showing the performance of a classification model at all classification thresholds. This curve plots two parameters: True Positive Rate and False Positive Rate.
**AUC** stands for ‘Area under the ROC Curve.’ That is, AUC measures the entire two-dimensional area underneath the entire ROC curve from (0,0) to (1,1), it provides an aggregate measure of performance across all possible classification thresholds. One way of interpreting AUC is as the probability that the model ranks a random positive example more highly than a random negative example.

In [8]:
%%time


#dictionary for collecting results
results_dict = {}

#obtaining default parameters by calling .fit() to XGBoost model instance
xgbc0 = xgb.XGBClassifier(objective='binary:logistic',
                          booster='gbtree',
                          eval_metric='auc',
                          tree_method='hist',
                          grow_policy='lossguide',
                          use_label_encoder=False)
xgbc0.fit(x_train , y_train)

#extracting default parameters from benchmark model
default_params = {}
gparams = xgbc0.get_params()

#default parameters have to be wrapped in lists - even single values - so GridSearchCV can take them as inputs
for key in gparams.keys():
    gp = gparams[key]
    default_params[key] = [gp]

#benchmark model. Grid search is not performed, since only single values are provided as parameter grid.
#However, cross-validation is still executed
clf0 = GridSearchCV(estimator=xgbc0, scoring='accuracy', param_grid=default_params, return_train_score=True, verbose=1, cv=3)
clf0.fit(x_train, y_train.values.ravel())

#results dataframe
df = pd.DataFrame(clf0.cv_results_)

#predictions - inputs to confusion matrix

train_predictions = clf0.predict(x_train)

print(train_predictions)

test_predictions = clf0.predict(x_test)





#confusion matrices
cfm_train = confusion_matrix(y_train, train_predictions)
print(cfm_train)
cfm_test = confusion_matrix(y_test, test_predictions)
print(cfm_test)


#accuracy scores
accs_train = accuracy_score(y_train, train_predictions)
print(accs_train)
accs_test = accuracy_score(y_test, test_predictions)
print(accs_test)


#F1 scores for each train/test label
f1s_train_p0 = f1_score(y_train, train_predictions, labels=[0], average=None)
print(f1s_train_p0)
f1s_train_p1 = f1_score(y_train, train_predictions, labels=[1], average=None)
print(f1s_train_p1)
f1s_train_p2 = f1_score(y_train, train_predictions, labels=[2], average=None)
print(f1s_train_p2)
f1s_test_p0 = f1_score(y_test, test_predictions, labels=[0], average=None)
print(f1s_test_p0)
f1s_test_p1 = f1_score(y_test, test_predictions, labels=[1], average=None)
print(f1s_test_p1)
f1s_test_p2 = f1_score(y_test, test_predictions, labels=[2], average=None)
print(f1s_test_p2)




labels = [0, 1, 2]

#Area Under the Receiver Operating Characteristic Curve clf0.predict_proba(x_test)[:,1]
pred_prob = clf0.predict_proba(x_test)
test_ras = roc_auc_score(y_test, pred_prob , average="macro", multi_class='ovo', labels = labels)
print(test_ras)


#best parameters
bp = clf0.best_params_

#storing computed values in results dictionary
results_dict['xgbc0'] = {'iterable_parameter': np.nan,
                         'classifier': deepcopy(clf0),
                         'cv_results': df.copy(),
                         'cfm_train': cfm_train,
                         'cfm_test': cfm_test,
                         
                         'train_accuracy': accs_train,
                         'test_accuracy': accs_test,
                         
                         'train F1-score label 2': f1s_train_p2,
                         'train F1-score label 1': f1s_train_p1,
                         'train F1-score label 0': f1s_train_p0,
                         'test F1-score label 2': f1s_test_p2,
                         'test F1-score label 1': f1s_test_p1,
                         'test F1-score label 0': f1s_test_p0,


                         'test roc auc score': test_ras,
                         
                         'best_params': bp}

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[1 2 2 ... 0 1 2]
[0 1 0 ... 1 0 1]
[[1099 1076  500]
 [ 936 1242  514]
 [ 951 1056  626]]
[[178 318 163]
 [289 198 154]
 [286 334  80]]
0.370875
0.228
[0.38827062]
[0.40949555]
[0.29300257]
[0.25212465]
[0.26559356]
[0.14585232]
0.36955718081313754
CPU times: user 11 s, sys: 200 ms, total: 11.2 s
Wall time: 11.2 s


**Gamma** ([0,∞], default=0 ): minimum loss reduction required to make a further partition on a leaf node of the tree; a node is split only when the resulting split gives a positive reduction in the loss function.

**Lambda** (default=1): L2 regularization term on weights, analogous to Ridge regression.

**N_estimators**([0,∞], default=100): number of trees in our ensemble.

**Alpha** (no bounds provided, default=0): L1 regularization term on weights (Lasso), when working with a large number of features, it might improve speed performances.

**Learning rate** ([0,1], default=0.3): determines the step size at each iteration while your model optimizes toward its objective. A low learning rate makes computation slower, and requires more rounds to achieve the same reduction in residual error as a model with a high learning rate. But it optimizes the chances to reach the best optimum.

**Maximum depth** ([0,∞], default=6): maximum depth of a tree. Increasing this value will make the model more complex, and might increase the performance, and more likely to overfit.

In [9]:
#creating deepcopy of default parameters before manipulations
params = deepcopy(default_params)

#setting grid of selected parameters for iteration
param_grid = {'gamma': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4, 200],
              'reg_lambda': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'n_estimators': [50,65,80,100,115,130,150],
              'reg_alpha': [0,0.1,0.2,0.4,0.8,1.6,3.2,6.4,12.8,25.6,51.2,102.4,200],
              'learning_rate': [0.01, 0.03, 0.06, 0.1, 0.15, 0.2, 0.25, 0.30, 0.4, 0.5, 0.6, 0.7],
              'max_depth': [5,6,7,8,9,10,11,12,13,14],
              }

Iterative Search

In [27]:
#start time
t0 = time.time()
#N of jobs
gcvj = np.cumsum([len(x) for x in param_grid.values()])[-1]

#iteration loop. Each selected parameter iterated separately
for i,grid_key in enumerate(param_grid.keys()):

    #variable for measuring iteration time
    loop_start = time.time()
       
    #creating param_grid argument for GridSearchCV:
    #listing grid values of current iterable parameter and wrapping non-iterable parameter single values in list
    for param_key in params.keys():
        if param_key == grid_key:
            params[param_key] = param_grid[grid_key]
        else:
            #use best parameters of last iteration
            try:
                param_value = [clf.best_params_[param_key]]
                params[param_key] = param_value
            #use benchmark model parameters for first iteration
            except:
                param_value = [clf0.best_params_[param_key]]
                params[param_key] = param_value
    
    #classifier instance of current iteration
    xgbc = xgb.XGBClassifier(**default_params)
    
    #GridSearch instance of current iteration
    clf = GridSearchCV(estimator=xgbc, param_grid=params, scoring='accuracy', return_train_score=True, verbose=1, cv=3)
    clf.fit(x_train, y_train.values.ravel())
    
    #results dataframe
    df = pd.DataFrame(clf.cv_results_)
    
    #predictions - inputs to confusion matrix
    train_predictions = clf.predict(x_train)
    test_predictions = clf.predict(x_test)
    

    #confusion matrices
    cfm_train = confusion_matrix(y_train, train_predictions)
    cfm_test = confusion_matrix(y_test, test_predictions)
    
    
    #accuracy scores
    accs_train = accuracy_score(y_train, train_predictions)
    accs_test = accuracy_score(y_test, test_predictions)
    
    
    #F1 scores for each train/test 
    f1s_train_p0 = f1_score(y_train, train_predictions, labels=[0], average=None)
    f1s_train_p1 = f1_score(y_train, train_predictions, labels=[1], average=None)
    f1s_train_p2 = f1_score(y_train, train_predictions, labels=[2], average=None)
    f1s_test_p0 = f1_score(y_test, test_predictions, labels=[0], average=None)
    f1s_test_p1 = f1_score(y_test, test_predictions, labels=[1], average=None)
    f1s_test_p2 = f1_score(y_test, test_predictions, labels=[2], average=None)
    
    
    
    
     
    labels = [0, 1, 2]

    #Area Under the Receiver Operating Characteristic Curve
    pred_prob = clf.predict_proba(x_test)
    test_ras = roc_auc_score(y_test, pred_prob , average="macro", multi_class='ovo', labels = labels)
    
    
    #best parameters
    bp = clf.best_params_
    
    #storing computed values in results dictionary
    results_dict[f'xgbc{i+1}'] = {'iterable_parameter': grid_key,
                                  'classifier': deepcopy(clf),
                                  'cv_results': df.copy(),
                                  'cfm_train': cfm_train,
                                  'cfm_test': cfm_test,
                                  
                                  'train_accuracy': accs_train,
                                  'test_accuracy': accs_test,
                                  
                                  'train F1-score label 2': f1s_train_p2,
                                  'train F1-score label 1': f1s_train_p1,
                                  'train F1-score label 0': f1s_train_p0,
                                  'test F1-score label 2': f1s_test_p2,
                                  'test F1-score label 1': f1s_test_p1,
                                  'test F1-score label 0': f1s_test_p0,
                                  
                                  
                                  'test roc auc score': test_ras,
                                  
                                  'best_params': bp}
    
    #variable for measuring iteration time
    elapsed_time = time.time() - loop_start
    print(f'iteration #{i+1} finished in: {elapsed_time} seconds')

#stop time
t1 = time.time()

#elapsed time
gcvt = t1 - t0

Fitting 3 folds for each of 13 candidates, totalling 39 fits
iteration #1 finished in: 73.59016013145447 seconds
Fitting 3 folds for each of 13 candidates, totalling 39 fits
iteration #2 finished in: 72.2410044670105 seconds
Fitting 3 folds for each of 7 candidates, totalling 21 fits
iteration #3 finished in: 75.25157475471497 seconds
Fitting 3 folds for each of 13 candidates, totalling 39 fits
iteration #4 finished in: 54.581161975860596 seconds
Fitting 3 folds for each of 12 candidates, totalling 36 fits
iteration #5 finished in: 73.15333271026611 seconds
Fitting 3 folds for each of 10 candidates, totalling 30 fits
iteration #6 finished in: 168.2721564769745 seconds


Randomized Search: randomly choosing hyperparameter values from the search space. Random point in the search space: probably not a minimum.





In [ ]:
%%time

#start time
t0 = time.time()

#No. of jobs
rcvj = gcvj

#unwrapping list values of default parameters
default_params_xgb = {}

for key in default_params.keys():
    default_params_xgb[key] = default_params[key][0]

#providing default parameters to xgbc model, before randomized search cross-validation
xgbc = xgb.XGBClassifier(**default_params_xgb)

#Executing Randomized Search
clf = RandomizedSearchCV(estimator=xgbc, param_distributions=param_grid, scoring='accuracy', return_train_score=True, verbose=1, cv=3, n_iter=rcvj)
clf.fit(x_train, y_train.values.ravel())
    
#results dataframe
df = pd.DataFrame(clf.cv_results_)
    
#predictions - inputs to confusion matrix
train_predictions = clf.predict(x_train)
test_predictions = clf.predict(x_test)

    
#confusion matrices
cfm_train = confusion_matrix(y_train, train_predictions)
cfm_test = confusion_matrix(y_test, test_predictions)
    
#accuracy scores
accs_train = accuracy_score(y_train, train_predictions)
accs_test = accuracy_score(y_test, test_predictions)

    
#F1 scores for each train/test label
f1s_train_p0 = f1_score(y_train, train_predictions, labels=[0], average=None)
f1s_train_p1 = f1_score(y_train, train_predictions, labels=[1], average=None)
f1s_train_p2 = f1_score(y_train, train_predictions, labels=[2], average=None)
f1s_test_p0 = f1_score(y_test, test_predictions, labels=[0], average=None)
f1s_test_p1 = f1_score(y_test, test_predictions, labels=[1], average=None)
f1s_test_p2 = f1_score(y_test, test_predictions, labels=[2], average=None)
    

labels = [0, 1, 2]

#Area Under the Receiver Operating Characteristic Curve
pred_prob = clf.predict_proba(x_test)
test_ras = roc_auc_score(y_test, pred_prob , average="macro", multi_class='ovo', labels = labels)
    
#best parameters
bp = clf.best_params_
    
#storing computed values in results dictionary
results_dict['xgbc_rcv'] = {'classifier': deepcopy(clf),
                            'cv_results': df.copy(),
                            'cfm_train': cfm_train,
                            'cfm_test': cfm_test,
                            
                            'train_accuracy': accs_train,
                            'test_accuracy': accs_test,
                            
                            'train F1-score label 2': f1s_train_p2,
                            'train F1-score label 1': f1s_train_p1,
                            'train F1-score label 0': f1s_train_p0,
                            'test F1-score label 2': f1s_test_p2,
                            'test F1-score label 1': f1s_test_p1,
                            'test F1-score label 0': f1s_test_p0,
                                  
                            
                            
                            'test roc auc score': test_ras,
                            
                            'best_params': bp}

#stop time
t1 = time.time()

#elapsed time
rcvt = t1 - t0


Fitting 3 folds for each of 68 candidates, totalling 204 fits


In [ ]:
#printing benchmark, iterative grid search and randomized search ROC AUC / accuracy scores (test data set)
print(f"Benchmark AUC/accuracy score: {np.round(results_dict['xgbc0']['test roc auc score'],4)} / {np.round(results_dict['xgbc0']['test_accuracy'],4)}")
print(f"Randomized search AUC/accuracy score: {np.round(results_dict['xgbc_rcv']['test roc auc score'],4)} / {np.round(results_dict['xgbc_rcv']['test_accuracy'],4)} @ {np.round(rcvt/60,2)} minutes and {rcvj} jobs")
print("")